In [1]:
devtools::install_github("NPSDC/beaveR", ref = "dev")
setwd('/fs/cbcb-lab/rob/students/noor/Uncertainity/treeTerminusScripts')
suppressPackageStartupMessages(source("tree_helper_function.R"))
suppressPackageStartupMessages(library(beaveR))
clustFile <- "../brain_sim_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/terminus/no_threshold0/cluster_nwk.txt"
quantDir <- file.path("../brain_sim_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/out_sal") 
samples <- as.vector(outer(c(1:6), c(1,2), function(x,y) paste(x,y,sep='_')))
quantFiles <- file.path(quantDir, samples, 'quant.sf')
coldata <- data.frame(files=quantFiles, names=samples, condition=factor(rep(1:2, each=6)))
# tse <- buildTSE(treeTermFile = clustFile, coldata = coldata)

Skipping install of 'beaveR' from a github remote, the SHA1 (55f74bc0) has not changed since last install.
  Use `force = TRUE` to force installation



In [2]:
saveDir <- "environment/brain_sim_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/comp_trees/DE"
load(file=file.path(saveDir, "tseCons.RData"))

In [3]:
temp <- tse

In [4]:
load("environment/brain_sim_nodtu/mode=gc_bias/simulate.rda")
suppressPackageStartupMessages(library(TreeSummarizedExperiment))
suppressPackageStartupMessages(library(ape))
tree <- rowTree(tse)
l <- length(tree$tip)
missingTxps <- setdiff(rownames(tse)[1:l], rownames(sim.counts.mat))
dts <- Reduce(union, c(names(iso.dge)[iso.dge], names(iso.dte)[iso.dte], names(iso.dte.only)[iso.dte.only], 
                       names(iso.dtu)[iso.dtu]))
dts <- intersect(dts, rownames(sim.counts.mat))
sim.counts.mat <- rbind(sim.counts.mat, matrix(0, nrow = length(missingTxps), ncol = ncol(sim.counts.mat),
                                               dimnames = list(missingTxps, colnames(sim.counts.mat))))
sim.counts.mat <- sim.counts.mat[rownames(tse)[1:l],]
dts <- intersect(dts, rownames(sim.counts.mat))

innNodes <- l+1:tree$Nnode
aggCountsNodes <- computeAggNodesU(tree, c(1:length(tree$tip),innNodes), sim.counts.mat)
logFCNodes <- ifelse(rowSums(aggCountsNodes)==0, 0, log2(aggCountsNodes[,2]+1)-log2(aggCountsNodes[,1]+1))
names(logFCNodes)[(l+1):length(logFCNodes)] <- paste("Node", names(logFCNodes)[(l+1):length(logFCNodes)], sep="")
rootFC <- abs(logFCNodes[l+1])


In [9]:
length(dts)
summary(abs(logFCNodes[dts]))
print(mean(abs(logFCNodes[dts]) >= rootFC))
rootFC
mcols(tse)[["meanInfRV"]][l+1]

[1] 8798

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.8858  1.5388  1.9330  1.8818  2.2746  2.5824 

[1] 1


Node138201 
0.03264371

Node138201 
      0.01

In [15]:
load(file.path(saveDir, "cutsCons.RData"))

#### Run Swish

In [10]:
set.seed(1)
y <- fishpond::scaleInfReps(temp[1:l,])
y <- labelKeep(y)
y <- swish(y, x="condition")

In [12]:
detNodes <- list()
detNodes[["Swish"]] <- lapply(c(0.01, 0.05, 0.1), function(x) which(mcols(y)[,"qvalue"] <= x ))

negNodes <- list()
negNodes[["Swish"]] <- lapply(detNodes[["Swish"]], function(nodes) setdiff(seq(l), 
                                                          unlist(Descendants(tree, nodes))))

#### Min meanInfRV and Height

In [18]:
lambda <- c(0.05, 0.1, 1, 5, 10)


[1] "irv_height_0.05" "irv_height_0.1"  "irv_height_1"    "lfcTPMDesc"     
[5] "LFC"             "lfcTPM"

In [16]:
cuts <- list()
for(i in lambda) {
    descSize <- sapply(Descendants(tree, seq(nrow(tse))), length)
    metric <- (mcols(temp)[['meanInfRV']] + node.depth(tree, 2)*i) *descSize
    objS <- solveForOptimalCut(tse, metVec = metric, type = 'min')
    gammaN <- paste("gamma",i,sep="=")
    cuts[[gammaN]] <- objS[["cut"]]
}
names(cuts) <- paste("irv", "height", lambda, sep = "_") 
sapply(cuts,length)

irv_height_0.05  irv_height_0.1    irv_height_1    irv_height_5   irv_height_10 
          76883           87912          119765          133437          136186

In [46]:
for(i in seq_along(lambda)) {
     ihname <- paste("irv", "height", lambda[i], sep = "_") 
     y <- temp[sort(cuts[[ihname]]),]
     y <- fishpond::scaleInfReps(y)
     y <- labelKeep(y)
     set.seed(1)
     y <- swish(y, x="condition")
#      dim(y)
     detNodes[[ihname]] <- lapply(c(0.01, 0.05, 0.1), function(x) {
        nodes <- rownames(y)[which(mcols(y)[,"qvalue"] <= x)]
#         txps <- match(nodes[grep("ENST", nodes)], rownames(tse))
#         iNodes <- match(nodes[grep("Node", nodes)], rownames(tse))
         match(nodes, names(logFCNodes))
        
#         c(txps,iNodes)
    })
    negNodes[[ihname]] <- lapply(c(0.01, 0.05, 0.1), function(x) {
        nodes <- rownames(y)[setdiff(seq(nrow(y)), which(mcols(y)[,"qvalue"] <= x))]
        match(nodes, names(logFCNodes))
    })
}

#### Max Weighted Log Fold Change

In [23]:
getLog2FC <- function(tse, condition, pc=5, array=FALSE) {
  infRepIdx <- grep("infRep", assayNames(tse))
  infReps <- assays(tse)[infRepIdx]
  infRepsArray <- abind::abind(as.list(infReps), along=3)
  dims <- dim(infRepsArray)
    
  cond1 <- condition == levels(condition)[1]
  cond2 <- condition == levels(condition)[2]

  diffs <- matrix(nrow=dims[1],ncol=dims[3])
  for (k in seq_len(dims[3])) {
    diffs[,k] <- log2(rowMeans(infRepsArray[,cond2,k]) + pc) -
                 log2(rowMeans(infRepsArray[,cond1,k]) + pc)
  }
  if (array) {
    return(diffs)
  }
  # median over inferential replicates
  rowMedians(diffs)
}

In [24]:
tse <- computeSizeFactors(tse, type="txp")
tse <- scaleInfReps(tse)
lfc <- getLog2FC(tse, colData(tse)[["condition"]])
descSize <- sapply(phangorn::Descendants(tree, seq(nrow(tse))), length)
metric <- abs(lfc)/mcols(tse)[['meanInfRV']]
objS <- solveForOptimalCut(tse, metVec = metric, type = 'max')
cuts[["LFC"]] <- objS[["cut"]]

Setting inf rep depth to mean depth



Progress:   1 on 100Progress:   2 on 100Progress:   3 on 100Progress:   4 on 100Progress:   5 on 100Progress:   6 on 100Progress:   7 on 100Progress:   8 on 100Progress:   9 on 100Progress:  10 on 100Progress:  11 on 100Progress:  12 on 100Progress:  13 on 100Progress:  14 on 100Progress:  15 on 100Progress:  16 on 100Progress:  17 on 100Progress:  18 on 100Progress:  19 on 100Progress:  20 on 100Progress:  21 on 100Progress:  22 on 100Progress:  23 on 100Progress:  24 on 100Progress:  25 on 100Progress:  26 on 100Progress:  27 on 100Progress:  28 on 100Progress:  29 on 100Progress:  30 on 100Progress:  31 on 100Progress:  32 on 100Progress:  33 on 100Progress:  34 on 100Progress:  35 on 100Progress:  36 on 100Progress:  37 on 100Progress:  38 on 100Progress:  39 on 100Progress:  40 on 100Progress:  41 on 100Progress:  42 on 100Progress:  43 on 100Progress:  44 on 100Progress:  45 on 100Progress:  46 on 100Progress:  47 on 100Progress:  48 on 100Progress:  49 on 100Progress:  50 on 100

In [63]:
y <- temp[sort(cuts[["LFC"]]),]
y <- fishpond::scaleInfReps(y)
y <- labelKeep(y)
set.seed(1)
y <- swish(y, x="condition")

detNodes[["LFC"]] <- lapply(c(0.01, 0.05, 0.1), function(x) {
    nodes <- rownames(y)[which(mcols(y)[,"qvalue"] <= x)]
    match(nodes, names(logFCNodes))
})
negNodes[["LFC"]] <- lapply(c(0.01, 0.05, 0.1), function(x) {
        nodes <- rownames(y)[setdiff(seq(nrow(y)), which(mcols(y)[,"qvalue"] <= x))]
        match(nodes, names(logFCNodes))
})

#### TPM LFC Desc

In [64]:
lfcTPM <- getScaledLFC(temp, "condition")
metric <- abs(lfcTPM)/mcols(tse)[['meanInfRV']]*descSize
objS <- solveForOptimalCut(tse, metVec = metric, type = 'max')
cuts[["lfcTPMDesc"]] <- objS[["cut"]]
y <- temp[sort(cuts[["lfcTPMDesc"]]),]
y <- fishpond::scaleInfReps(y)
y <- labelKeep(y)
set.seed(1)
y <- swish(y, x="condition")

detNodes[["lfcTPMDesc"]] <- lapply(c(0.01, 0.05, 0.1), function(x) {
    nodes <- rownames(y)[which(mcols(y)[,"qvalue"] <= x)]
    match(nodes, names(logFCNodes))
})
negNodes[["lfcTPMDesc"]] <- lapply(c(0.01, 0.05, 0.1), function(x) {
        nodes <- rownames(y)[setdiff(seq(nrow(y)), which(mcols(y)[,"qvalue"] <= x))]
        match(nodes, names(logFCNodes))
    })

#### TPM LFC

In [65]:
lfcTPM <- getScaledLFC(temp, "condition")
metric <- abs(lfcTPM)/mcols(tse)[['meanInfRV']]
objS <- solveForOptimalCut(tse, metVec = metric, type = 'max')
cuts[["lfcTPM"]] <- objS[["cut"]]
y <- temp[sort(cuts[["lfcTPM"]]),]
y <- fishpond::scaleInfReps(y)
y <- labelKeep(y)
set.seed(1)
y <- swish(y, x="condition")

detNodes[["lfcTPM"]] <- lapply(c(0.01, 0.05, 0.1), function(x) {
    nodes <- rownames(y)[which(mcols(y)[,"qvalue"] <= x)]
    match(nodes, names(logFCNodes))
})
negNodes[["lfcTPM"]] <- lapply(c(0.01, 0.05, 0.1), function(x) {
        nodes <- rownames(y)[setdiff(seq(nrow(y)), which(mcols(y)[,"qvalue"] <= x))]
        match(nodes, names(logFCNodes))
    })

In [73]:
df <- data.frame(Method=character(), Metric=character(), FDR_0.01 = numeric(), FDR_0.05 = numeric(), FDR_0.10 = numeric())
for(n in names(detNodes))
{
    if(n!="term") {
        print(n)
        stats <-  sapply(seq_along(detNodes[[n]]), function(i) computeMetOut(detNodes[[n]][[i]], logFCNodes, tree = tree, negNodes = negNodes[[n]][[i]], lfcThresh = rootFC))
        colnames(stats) <- colnames(df)[c(3:5)]
        stats <- cbind(Method=rep(n,2), Metric = c("FDR", "TPR"), stats)
        df <- rbind(df, stats)
        
    }
    
}
print(df)

[1] "Swish"
[1] "tp 0.990278047832005"
[1] "tp 0.962566844919786"
[1] "tp 0.9264664608476"
[1] "irv_height_0.05"
[1] "tp 0.989345032905045"
[1] "tp 0.958007303077726"
[1] "tp 0.915940663998116"
[1] "irv_height_0.1"
[1] "tp 0.989531840651352"
[1] "tp 0.962882096069869"
[1] "tp 0.925461741424802"
[1] "irv_height_1"
[1] "tp 0.998256624825662"
[1] "tp 0.964686407945558"
[1] "tp 0.923314378554021"
[1] "irv_height_5"
[1] "tp 0.997685950413223"
[1] "tp 0.963011031797534"
[1] "tp 0.920902341519132"
[1] "irv_height_10"
[1] "tp 0.990298797050834"
[1] "tp 0.956297799282035"
[1] "tp 0.919503348214286"
[1] "LFC"
[1] "tp 0.990543014032947"
[1] "tp 0.962216624685139"
[1] "tp 0.92042620720925"
[1] "lfcTPMDesc"
[1] "tp 0.998256624825662"
[1] "tp 0.964686407945558"
[1] "tp 0.923314378554021"
[1] "lfcTPM"
[1] "tp 0.989514679448772"
[1] "tp 0.962062980411604"
[1] "tp 0.920517972761777"
              Method Metric    FDR_0.01   FDR_0.05   FDR_0.10
fdr            Swish    FDR 0.009721952 0.03743316 0.073533

#### Parse Terminus

In [57]:
suppressPackageStartupMessages(source("old_terminus_helper.R"))
dir <- "/fs/cbcb-lab/rob/students/noor/Uncertainity/brain_sim_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/terminus/"
seBrainSim <- tximeta(coldata)
yTermThrNS <- seBrainSim[rownames(tse)[1:l],]
groupsClust <- parseClustFile(file.path(dir, "old", "1_1", "clusters.txt"), yTermThrNS)
mInds <- seq(nrow(yTermThrNS) + length(groupsClust))
yAggTermThrNS <- prepOSwish(yTermThrNS, mInds, groupsClust)

rownames(yAggTermThrNS)[l+1:length(groupsClust)] <- paste("Node", as.character(l+1:length(groupsClust)),sep="")
metadata(yAggTermThrNS)[["txpsAnn"]] <- rowData(yTermThrNS)
yAggTermThrNS <- yAggTermThrNS[-unlist(groupsClust),]
dim(yAggTermThrNS)

importing quantifications

reading in files with read_tsv

1 
2 
3 
4 
5 
6 
7 
8 
9 
10 
11 
12 


found matching transcriptome:
[ GENCODE - Homo sapiens - release 26 ]

loading existing TxDb created: 2022-08-19 22:12:59

loading existing transcript ranges created: 2022-08-19 22:13:01



[1] 129988     12

In [55]:
aggCountsNodes <- computeOAggNodesU(groupsClust, mInds, sim.counts.mat)
logFCNodesTerm <- ifelse(rowSums(aggCountsNodes)==0, 0, log2(aggCountsNodes[,2]+1)-log2(aggCountsNodes[,1]+1))
names(logFCNodesTerm)[(l+1):length(logFCNodesTerm)] <- paste("Node", names(logFCNodesTerm)[(l+1):length(logFCNodesTerm)], sep="")


In [58]:
yAggTermThrNS <- fishpond::scaleInfReps(yAggTermThrNS)
yAggTermThrNS <- labelKeep(yAggTermThrNS)
set.seed(1)
yAggTermThrNS <- swish(yAggTermThrNS, x="condition")

In [74]:
detNodes[["term"]] <- lapply(c(0.01, 0.05, 0.1), function(x) {
        nodes <- rownames(yAggTermThrNS)[which(mcols(yAggTermThrNS)[,"qvalue"] <= x)]
        match(nodes, names(logFCNodesTerm))
    })
negNodes[["term"]] <- lapply(c(0.01, 0.05, 0.1), function(x) {
        nodes <- rownames(yAggTermThrNS)[setdiff(seq(nrow(yAggTermThrNS)),
                                                 which(mcols(yAggTermThrNS)[,"qvalue"] <= x))]
        match(nodes, names(logFCNodesTerm))
    })
stats <-  sapply(seq_along(detNodes[["term"]]), function(i) computeMetOut(detNodes[["term"]][[i]], 
                                                                          logFCNodesTerm, tree = NULL, 
                                                                          negNodes = negNodes[["term"]][[i]],
                                                                          lfcThresh = rootFC))
colnames(stats) <- colnames(df)[c(3:5)]
stats <- cbind(Method=rep("term",2), Metric = c("FDR", "TPR"), stats)
df <- rbind(df, stats)
# detNodes[["term"]] <- lapply(c(0.01, 0.05, 0.1), function(x) {
#     nodes <- which(mcols(yAggTermThrNS)[,"qvalue"] <= x)
#     tgroups <- nodes[nodes > l]
#     groupTxps <- unlist(groupsClust) ## Transcripts covered by the groups
#     txps <- intersect(nodes, setdiff(seq(l), groupTxps))
#     c(txps, tgroups)
# })
# negNodes[["term"]] <- lapply(detNodes[["term"]], function(nodes) {
#     groupTxps <- unlist(groupsClust)
#     ndnodes <- setdiff(l+(1:length(groupsClust)), nodes)
#     ndtxps <- setdiff(seq(l), c(nodes, groupTxps))
#     c(ndtxps, ndnodes)
# })

[1] "tp 0.990955806783145"
[1] "tp 0.962254738942468"
[1] "tp 0.924368499257058"


In [75]:
df

,Method,Metric,FDR_0.01,FDR_0.05,FDR_0.10
,<named list>,<named list>,<named list>,<named list>,<named list>
fdr,Swish,FDR,0.009721952,0.03743316,0.07353354
tpr,Swish,TPR,0.5788816,0.6956126,0.7503978
fdr1,irv_height_0.05,FDR,0.01065497,0.0419927,0.08405934
tpr1,irv_height_0.05,TPR,0.6280088,0.7306545,0.7738214
fdr2,irv_height_0.1,FDR,0.01046816,0.0371179,0.07453826
tpr2,irv_height_0.1,TPR,0.6233742,0.7270562,0.7710203
fdr3,irv_height_1,FDR,0.001743375,0.03531359,0.07668562
tpr3,irv_height_1,TPR,0.3916017,0.7174121,0.7773218
fdr4,irv_height_5,FDR,0.00231405,0.03698897,0.07909766


In [76]:
saveDir <- "environment/brain_sim_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/comp_trees/DE"
save(cuts, file=file.path(saveDir, "cutsCons.RData"))
save(df, file=file.path(saveDir, "dfCons.RData"))

In [77]:
negNodesDesc <- list()
for(n in names(negNodes)) {
    if(n == "Swish")
        negNodesDesc[[n]] <- negNodes[[n]]
    if(n != "term")
        negNodesDesc[[n]] <- lapply(negNodes[[n]], function(x) unlist(Descendants(tree, x)))
}
negNodesDesc[["term"]] <- lapply(negNodes[["term"]], function(x) {
    dd <- x[x>l]
    r <- setdiff(x, dd)
    dd <- dd - l
    c(unlist(groupsClust[dd]), r) 
})

In [26]:
sapply(negNodes[["lfcTPM"]],length)
sapply(negNodesDesc[["lfcTPM"]],length)


[1] 83067 82372 81926

[1] 70733 70332 70089

In [78]:
dfO <- data.frame(Method=character(), Metric=character(), FDR_0.01 = numeric(), FDR_0.05 = numeric(), FDR_0.10 = numeric())
for(n in names(detNodes))
{
    print(n)
    if(n != "term") {
        stats <-  sapply(seq_along(detNodes[[n]]), function(i) computeMetOut(detNodes[[n]][[i]], 
                                                                             logFCNodes, tree = tree, 
                                                                             negNodes = negNodesDesc[[n]][[i]], 
                                                                             lfcThresh = rootFC))
        colnames(stats) <- colnames(df)[c(3:5)]
        stats <- cbind(Method=rep(n,2), Metric = c("FDR", "TPR"), stats)
        dfO <- rbind(dfO, stats)
    }
    
}
stats <-  sapply(seq_along(detNodes[["term"]]), function(i) computeMetOut(detNodes[["term"]][[i]], 
                                              logFCNodesTerm, tree = NULL, 
                                              negNodes = negNodesDesc[["term"]][[i]],
                                              lfcThresh = rootFC))
colnames(stats) <- colnames(df)[c(3:5)]
stats <- cbind(Method=rep("term",2), Metric = c("FDR", "TPR"), stats)
dfO <- rbind(dfO, stats)

print(dfO)

[1] "Swish"
[1] "tp 0.990278047832005"
[1] "tp 0.962566844919786"
[1] "tp 0.9264664608476"
[1] "irv_height_0.05"
[1] "tp 0.989345032905045"
[1] "tp 0.958007303077726"
[1] "tp 0.915940663998116"
[1] "irv_height_0.1"
[1] "tp 0.989531840651352"
[1] "tp 0.962882096069869"
[1] "tp 0.925461741424802"
[1] "irv_height_1"
[1] "tp 0.998256624825662"
[1] "tp 0.964686407945558"
[1] "tp 0.923314378554021"
[1] "irv_height_5"
[1] "tp 0.997685950413223"
[1] "tp 0.963011031797534"
[1] "tp 0.920902341519132"
[1] "irv_height_10"
[1] "tp 0.990298797050834"
[1] "tp 0.956297799282035"
[1] "tp 0.919503348214286"
[1] "LFC"
[1] "tp 0.990543014032947"
[1] "tp 0.962216624685139"
[1] "tp 0.92042620720925"
[1] "lfcTPMDesc"
[1] "tp 0.998256624825662"
[1] "tp 0.964686407945558"
[1] "tp 0.923314378554021"
[1] "term"
[1] "lfcTPM"
[1] "tp 0.989514679448772"
[1] "tp 0.962062980411604"
[1] "tp 0.920517972761777"
[1] "tp 0.990955806783145"
[1] "tp 0.962254738942468"
[1] "tp 0.924368499257058"
              Method Metric  

In [79]:
save(dfO, file=file.path(saveDir, "dfConsO.RData"))